<a href="https://colab.research.google.com/github/van-dang/MRI-Cloud/blob/master/StrongPeriodicBC_TwoComp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#About the solver

In [0]:
# This demo solves the transformed Bloch-Torrey equation applied to diffusion MRI 
# using the standard finite element method coupled with the theta-method 
# for the space discretization.

# This demo allows for a two-compartment domain with periodic boundaries and the 
# spins can enter and exit the computational domain freely by using the periodic 
# boundary conditions

# Copyright (C) 2017 Van-Dang Nguyen

# This file is part of DOLFIN.

# DOLFIN is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# DOLFIN is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
# GNU Lesser General Public License for more details.

# You should have received a copy of the GNU Lesser General Public License
# along with DOLFIN. If not, see <http://www.gnu.org/licenses/>.

# First added:  2017-10-10
# Last changed: 2019-04-10

# Setting a working environment

In [2]:
from google.colab import files

import platform, sys
python_version=platform.python_version()
from distutils.version import LooseVersion, StrictVersion

if ( LooseVersion(python_version) < LooseVersion("3.0.0")):
    print("Python3 is needed!");
    print("How to fix: Runtime/Change_runtime_type/Python 3");
    sys.exit()
    
try:
    from dolfin import *; from mshr import *
except ImportError as e:
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:fenics-packages/fenics
    !apt-get update -qq
    !apt install -y --no-install-recommends fenics
    from dolfin import *; from mshr import *
    

import matplotlib.pyplot as plt;
from IPython.display import clear_output, display; import time; import dolfin.common.plotting as fenicsplot 
import time

import os, sys, shutil

clear_output(); # Plotting setup

dolfin_version = dolfin.__version__
print ('dolfin version:', dolfin_version)


!rm -rf * # clean up all files

# dolfin setting
q_degree = 3
dx = dx(metadata={'quadrature_degree': q_degree})

# Useful commands
# Remove an empty folder      : os.rmdir("my_results")
# Remove a folder with files  : shutil.rmtree("results")
# Make a folder               : os.mkdir("my_results")
# Runtime/Change_runtime_type/Python3

dolfin version: 2018.1.0


In [0]:
Nsteps = 100;
bvalue = 1000;

delta, Delta = 50000, 50000


gnorm = sqrt(bvalue)/sqrt(delta*delta*(Delta-delta/3.0));
print("Nsteps: ", Nsteps, "bvalue: ",bvalue)


def SubMeshSave(ur, ui, file_ur, file_ui, mesh, n, stepcounter, dolfin_version):
  if dolfin_version=='1.6.0':
    V = FunctionSpace(mesh, "CG", porder)
  else:
    # For FEniCS 2016, 2017
    Ve = FiniteElement("CG", mesh.ufl_cell(), porder)
    V = FunctionSpace(mesh, Ve)
  if stepcounter % n == 0:
    ur_p = project(ur, V);
    ui_p = project(ui, V);
    ur_p.rename("Real", "label");
    ui_p.rename("Imag", "label");
    file_ur << ur_p;
    file_ui << ui_p;

def ieval(u,omega, phase):
  if omega==1:
    return u('+')*phase('+') + u('-')*phase('-');
  if omega==0:
    return u('+')*(1.-phase('+')) + u('-')*(1.-phase('-'))                              

#####################################################################################################################
#####################################################################################################################
# GEOMETRY SETTINGS
xmin, ymin, zmin, xmax, ymax, zmax = -10, -10, -10, 10, 10, 10


mesh=Mesh('mesh2comp_R9_periodic.xml');

hmin = mesh.hmin();
center=[];
center.append([-4,0,0]);
center.append([16,0,0]);
R0=9;

print(center[0][0], center[0][1], center[0][2])
print(center[1][0], center[1][1], center[1][2])

class Exterior(SubDomain):
    def inside (self,x,on_boundary):
        d0 = sqrt((x[0]-center[0][0])*(x[0]-center[0][0])+(x[1]-center[0][1])*(x[1]-center[0][1])+(x[2]-center[0][2])*(x[2]-center[0][2]));
        d1 = sqrt((x[0]-center[1][0])*(x[0]-center[1][0])+(x[1]-center[1][1])*(x[1]-center[1][1])+(x[2]-center[1][2])*(x[2]-center[1][2]));
        return (d0>=R0-hmin/4.0 and d1>=R0-hmin/4.0);

class Interior(SubDomain):
    def inside (self,x,on_boundary):
        exterior = Exterior()
        return not(exterior.inside(x, True))

# Mark exterior domain to 1, interior domain to 0
exterior = Exterior();
interior = Interior();
cellmarker = MeshFunction("size_t", mesh, mesh.topology().dim())
cellmarker.set_all(0)
exterior.mark(cellmarker,1)

# plot(cellmarker); interactive(); stop
# stop;

mesh0 = SubMesh(mesh, cellmarker, 0)
mesh1 = SubMesh(mesh, cellmarker, 1)
V_DG = FunctionSpace(mesh, 'DG', 0)
dofmap_DG = V_DG.dofmap()
phase = Function(V_DG)
vol = CellVolume(mesh)
h = 0.5*CellDiameter(mesh);

for cell in cells(mesh):
    phase.vector()[dofmap_DG.cell_dofs(cell.index())] = cellmarker[cell.index()];

# plot(phase); interactive(); stop
mesh_file = File("mesh.xml")
mesh_file << mesh
phase_file = File("Phi.xml")
phase_file << phase
########## END OF GEOMETRY SETTINGS
#####################################################################################################################

# parameters
#####################################################################################################################
#####################################################################################################################
porder = 1;

# delta, Delta = 50000, 50000
t, T = 0, Delta+delta;
K0, K1 = 3e-3, 3e-3;
g0, g1, g2 = 1.0, 0.0, 0.0

kappa = 1e-5; # permeability
# kappa = 100.0; # permeability

nskip = 2;

# bvalue = 1000;
# gnorm = sqrt(bvalue)/sqrt(delta*delta*(Delta-delta/3.0));
#####################################################################################################################
#####################################################################################################################
# Sub domain for Periodic boundary condition
tol = 1e-7*mesh.hmin();
class PeriodicBoundary(SubDomain):
    # Left boundary is "target domain" G
    def inside(self, x, on_boundary):
        return bool(x[0] < xmin + tol and x[0] > xmin - tol and on_boundary)

    # Map right boundary (H) to left boundary (G)
    def map(self, x, y):
        y[0] = x[0] - (xmax-xmin)
        y[1] = x[1]
        y[2] = x[2]
#####################################################################################################################
#####################################################################################################################
# FUNCTION SPACES
if dolfin_version=='1.6.0':
    V = FunctionSpace(mesh , "CG", 1, constrained_domain=PeriodicBoundary());
    # V = FunctionSpace(mesh , "CG", 1);
    W = MixedFunctionSpace([V, V, V, V])
    VV = MixedFunctionSpace([V, V])
else:
    # For FEniCS 2016, 2017
    Ve = FiniteElement("CG", mesh.ufl_cell(), 1)
    TH = MixedElement([Ve,Ve,Ve,Ve])
    VVe = MixedElement([Ve,Ve])
    V = FunctionSpace(mesh,Ve);
    VV = FunctionSpace(mesh,VVe);
    W = FunctionSpace(mesh, TH, constrained_domain=PeriodicBoundary())

v = TestFunction(W)
v0r, v0i, v1r, v1i = v[0], v[1], v[2], v[3]

w = TrialFunction(W);
u0r, u0i, u1r, u1i = w[0], w[1], w[2], w[3]
#####################################################################################################################
#####################################################################################################################
# Initial conditions
# one = Function(V);
Dirac_Delta = Expression("x[0]*x[0]+x[1]*x[1]+x[2]*x[2]<eps",eps=1e6, domain=mesh, degree=2);
Dirac_Delta = interpolate(Dirac_Delta, V);
# plot(Dirac_Delta); interactive(); stop;
# one.vector()[:] = 1;
u_0 = Function(W);
assign(u_0.sub(0), Dirac_Delta)
assign(u_0.sub(2), Dirac_Delta)
u0r_0, u0i_0, u1r_0, u1i_0 = u_0[0], u_0[1], u_0[2], u_0[3]
#####################################################################################################################
#####################################################################################################################

g = Expression(("g0","g1","g2"), g0=g0, g1=g1, g2=g2,domain=mesh,degree=2);

#####################################################################################################################
#####################################################################################################################
## output files 
file_u0r = File("results/u0r.pvd")
file_u0i = File("results/u0i.pvd")
file_u1r = File("results/u1r.pvd")
file_u1i = File("results/u1i.pvd")
#####################################################################################################################
#####################################################################################################################
def FT(t, delta, Delta):
    ft1 = 1.0*(t>=0 and t<delta) 
    ft2 = -1.0*(t>=Delta and t<=Delta+delta);
    return ft1 + ft2;  

def iFT(t, delta, Delta): # integrate ft
    ft1 = t*(t>=0 and t<delta) 
    ft2 = delta*(t>=delta and t<Delta) 
    ft3 = (delta - t + Delta)*(t>=Delta and t<=Delta+delta) 
    return ft1 + ft2 + ft3;  

stepcounter = 0;
####################################################################################################################
####################################################################################################################
def FuncF(ift, gnorm, g, ur, ui, vr, vi, K):
    Fr =   ift*gnorm*(inner(g,K*grad(ui))+inner(grad(ui),K*g))*vr - inner(g,K*g)*gnorm*gnorm*ift*ift*ur*vr-K*inner(grad(ur), grad(vr))
    Fi = - ift*gnorm*(inner(g,K*grad(ur))+inner(grad(ur),K*g))*vi - inner(g,K*g)*gnorm*gnorm*ift*ift*ui*vi-K*inner(grad(ui), grad(vi))
    return Fr + Fi

def icondition(kappa, u0rm, u1rm, v0r, v1r, u0im, u1im, v0i, v1i):
    F_bcr = kappa*(u0rm-u1rm)*(v0r-v1r)
    F_bci = kappa*(u0im-u1im)*(v0i-v1i)
    return F_bcr + F_bci

def inner_interface(kappa, u0rm, u1rm, v0r, v1r, u0im, u1im, v0i, v1i, n, g, K0, K1):
    '''
    F_bcr  = (kappa*avg(u0rm-u1rm)+0.5*gnorm*ift*(avg(u0im)*inner(K0*avg(g),n)-avg(u1im)*inner(K1*avg(g),n)))*avg(v0r-v1r)
    F_bcr += -gnorm*ift*( avg(u0im)*inner(K0*avg(g),n)+ avg(u1im)*inner(K1*avg(g),n) )*0.5*avg(v0r+v1r)

    F_bci  = (kappa*avg(u0im-u1im)-0.5*gnorm*ift*(avg(u0rm)*inner(K0*avg(g),n)-avg(u1rm)*inner(K0*avg(g),n)))*avg(v0i-v1i)
    F_bci += gnorm*ift*(  avg(u0rm)*inner(K0*avg(g),n)+ avg(u1rm)*inner(K1*avg(g),n) )*0.5*avg(v0i+v1i)
    '''

    F_bcr  = (-kappa*avg(u0rm-u1rm)-0.5*gnorm*ift*(avg(u0im)*inner(K0*avg(g),n)+avg(u1im)*inner(K1*avg(g),n)))*avg(v0r-v1r)                      
    F_bcr += -gnorm*ift*( avg(u0im)*inner(K0*avg(g),n)-avg(u1im)*inner(K1*avg(g),n) )*0.5*avg(v0r+v1r)                                         
                                                                                                                                                
    F_bci  = (-kappa*avg(u0im-u1im)+0.5*gnorm*ift*(avg(u0rm)*inner(K0*avg(g),n)+avg(u1rm)*inner(K0*avg(g),n)))*avg(v0i-v1i)                      
    F_bci += gnorm*ift*(  avg(u0rm)*inner(K0*avg(g),n)-avg(u1rm)*inner(K1*avg(g),n) )*0.5*avg(v0i+v1i)                                         


    return -F_bcr - F_bci

def outer_interface(ift, gnorm, K, n, ur, ui, vr, vi, g):
    F_bcr =  (ift*gnorm+1e-16)*inner(K*g, n)*ui*vr
    F_bci = -(ift*gnorm+1e-16)*inner(K*g, n)*ur*vi
    return F_bcr + F_bci

#############################################################################################################################
#############################################################################################################################
def ThetaMethod_L(ift, gnorm, g, u0r, u0i, v0r, v0i, u1r, u1i, v1r, v1i, u0r_0, u0i_0, u1r_0, u1i_0,k, kappa, K0, K1, theta, phase,n, n0):
    L0 = (u0r_0/k*v0r + u0i_0/k*v0i +theta*FuncF(ift, gnorm, g, u0r_0, u0i_0, v0r, v0i, K0))*(1-phase)*dx
    L1 = (u1r_0/k*v1r + u1i_0/k*v1i +theta*FuncF(ift, gnorm, g, u1r_0, u1i_0, v1r, v1i, K1))*phase*dx
    L_inner_bc  = -(theta*inner_interface(kappa, u0r_0, u1r_0, v0r, v1r, u0i_0, u1i_0, v0i, v1i, n0, g, K0, K1))*abs(jump(phase))*dS;
    L0_outer_bc = -theta*outer_interface(ift, gnorm, K0, n, u0r_0, u0i_0, v0r, v0i, g)*ds
    L1_outer_bc = -theta*outer_interface(ift, gnorm, K1, n, u1r_0, u1i_0, v1r, v1i, g)*ds
    return L0+L1+L_inner_bc + L0_outer_bc+L1_outer_bc


def ThetaMethod_a(ift, gnorm, g, u0r, u0i, v0r, v0i, u1r, u1i, v1r, v1i, u0r_0, u0i_0, u1r_0, u1i_0,k,kappa, K0, K1, theta, phase,n, n0):
    a0 = (u0r/k*v0r   + u0i/k*v0i  -theta*FuncF(ift, gnorm, g, u0r  , u0i  , v0r, v0i, K0))*(1-phase)*dx
    a1 = (u1r/k*v1r   + u1i/k*v1i  -theta*FuncF(ift, gnorm, g, u1r  , u1i  , v1r, v1i, K1))*phase*dx
    a_inner_bc  = (  (theta*inner_interface(kappa, u0r  , u1r  , v0r, v1r, u0i  , u1i  , v0i, v1i, n0, g, K0, K1)))*abs(jump(phase))*dS;
    a0_outer_bc = theta*outer_interface(ift, gnorm ,K0, n, u0r, u0i, v0r, v0i, g)*ds
    a1_outer_bc = theta*outer_interface(ift, gnorm ,K1, n, u1r, u1i, v1r, v1i, g)*ds
    return a0+a1+a_inner_bc + a0_outer_bc + a1_outer_bc
#############################################################################################################################
#############################################################################################################################
# Theta method

stepcounter = 0;
theta = 0.5;
Nsteps = 50
k = T/Nsteps;

n = FacetNormal(mesh);
n0 = ieval(n, 0, phase);

while t < T + k: # Time-stepping loop
    if stepcounter % nskip == 0:
        print('t=%f '%t, 'gnorm: %f '%gnorm,'kappa:%f '%kappa);

    ft = FT(t, delta, Delta);
    ift = iFT(t, delta, Delta);

    L = ThetaMethod_L(ift, gnorm, g, u0r, u0i, v0r, v0i, u1r, u1i, v1r, v1i, u0r_0, u0i_0, u1r_0, u1i_0,k, kappa, K0, K1, theta, phase,n, n0);
    a = ThetaMethod_a(ift, gnorm, g, u0r, u0i, v0r, v0i, u1r, u1i, v1r, v1i, u0r_0, u0i_0, u1r_0, u1i_0,k, kappa, K0, K1, theta, phase,n, n0);
    a = a + 1e-7*(u0r*v0r+u0i*v0i)*theta*dx + 1e-7*(u1r*v1r+u1i*v1i)*(1-theta)*dx
    A = assemble(a);
    b = assemble(L);
 
    u = Function(W)
    solve(A,u.vector(),b, "gmres", "ilu");
    # solve(A,u.vector(),b, "lu");
    
    u0r_0, u0i_0, u1r_0, u1i_0 = split(u)
    
    # SubMeshSave(u0r_0, u0i_0, file_u0r, file_u0i, mesh0, nskip, stepcounter, dolfin_version);
    # SubMeshSave(u1r_0, u1i_0, file_u1r, file_u1i, mesh1, nskip, stepcounter, dolfin_version);
    
    t += k;
    stepcounter += 1;
plot(u0r_0); plot(u1r_0); interactive();
signal = assemble((phase*u1r_0+(1-phase)*u0r_0)*dx)/assemble(Dirac_Delta*dx);
print('b:',bvalue, 'Signal: %.3e'%signal)

Nsteps:  100 bvalue:  1000
-4 0 0
16 0 0
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
t=0.000000  gnorm: 0.000003  kappa:0.000010 
Calling FFC just-in-time (JIT) compiler, this may take some time.


/usr/lib/python3/dist-packages/ffc/uflacs/analysis/dependencies.py:61: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  active[targets] = 1


Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
